In [46]:
%pip install torchvision
%pip install torch
%pip install pandas


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os

device = torch.device("mps")
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: mps


In [9]:
from enum import Enum


class Dataset_type(Enum):
    TRAIN = 0
    TEST = 1


class CovidDataset(Dataset):
    def __init__(self, src_path, trg_file, dataset_type: Dataset_type, transform=None):
        super().__init__()
        self.src_path = src_path
        self.transform = transform
        self.dataset_type = dataset_type
        self.trg_df = pd.read_csv(trg_file)

    def __len__(self):
        return len(self.trg_df)

    def __getitem__(self, idx):
        filename = self.trg_df.iloc[idx, 0]
        src = Image.open(os.path.join(self.src_path, filename))
        src = self.transform(src) if self.transform is not None else src

        if self.dataset_type == Dataset_type.TRAIN:
            trg = torch.tensor(
                1 if self.trg_df.iloc[idx, 1] == "covid" else 0)

            return src, trg
        else:
            return src


In [32]:
current_path = os.getcwd()
img_path = os.path.join(current_path, 'train')
img_path

'/Users/lee/kdt-ai-course/competition/KDT_AI_project_competition_study/d반/[5기-A]이의진/1_corona/train'

In [10]:
from torch.utils.data import random_split

current_path = os.getcwd()
train_path = os.path.join(current_path, "dataset", "train")
train_trg_file = os.path.join(train_path, "labels.csv")
test_path = os.path.join(current_path, "dataset", "test")
test_trg_file = os.path.join(test_path, "submission.csv")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (1,))
])

train_set = CovidDataset(src_path=train_path, trg_file=train_trg_file,
                         dataset_type=Dataset_type.TRAIN, transform=transform)
test_set = CovidDataset(src_path=test_path, trg_file=test_trg_file,
                        dataset_type=Dataset_type.TEST, transform=transform)

train_valid_ratio = 0.9
train_set_count = int(len(train_set) * train_valid_ratio)
val_set_count = len(train_set) - train_set_count
train_set, val_set = random_split(train_set, [train_set_count, val_set_count])
print(len(train_set))
print(len(val_set))
print(len(test_set))

1800
200
400


In [34]:
train_set[0]

(tensor([[[-0.3902, -0.4333, -0.4686,  ..., -0.2098,  0.0137,  0.1863],
          [-0.4765, -0.4882, -0.4961,  ..., -0.3667, -0.2020, -0.0333],
          [-0.5000, -0.5000, -0.5000,  ..., -0.4569, -0.3471, -0.2137],
          ...,
          [-0.4961, -0.5000, -0.5000,  ..., -0.1784, -0.1549, -0.1549],
          [-0.4922, -0.5000, -0.5000,  ..., -0.1431, -0.1235, -0.1314],
          [-0.4882, -0.4961, -0.5000,  ..., -0.1118, -0.0922, -0.1118]]]),
 tensor(0))

In [11]:
from torch.utils.data import DataLoader
# 데이터로더를 만듭니다.
batch_size = 32
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True)


In [7]:
class Conv_Net(nn.Module):
    def __init__(self):
        super(Conv_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.maxpool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64*74*74, 128)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.maxpool(x)
        x = torch.relu(self.conv2(x))
        x = self.maxpool(x)
        x = x.view(-1, 64 * 74 * 74)  # 입력 데이터를 펼칩니다.
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return torch.sigmoid(x)


model = Conv_Net().to(device)

# 손실 함수와 최적화 알고리즘을 정의합니다.
criterion = nn.BCELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)


def train(num_of_epoch):
    train_losses = []
    train_accs = []
    val_losses = []
    val_accs = []

    model.train()
    correct = 0
    for epoch in range(num_of_epoch):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            import sys

            # print(f"target: {target.shape}")
            # print(f"output: {output.shape}")
            output = torch.squeeze(output, dim=1)
            output = output.to(torch.float32)
            target = target.to(torch.float32)
            # print(f"target: {target.shape}")
            # print(f"output: {output.shape}")
            # print(target)
            # print(output)
            # sys.exit()
            loss = criterion(output, target)
            # sys.exit()
            loss.backward()
            optimizer.step()
            pred = torch.tensor(
                [1 if output_data >= 0.5 else 0 for output_data in output]).to(device)
            correct += pred.eq(target.view_as(pred)).sum().item()

            if batch_idx % 5 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch + 1, batch_idx *
                    len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()))

    train_accuracy = 100. * (correct / len(train_set))
    train_losses.append(loss.item())
    train_accs.append(train_accuracy)

    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            output = torch.squeeze(output, dim=1)
            output = output.to(torch.float32)
            target = target.to(torch.float32)
            loss = criterion(output, target)
            pred = torch.tensor(
                [1 if output_data >= 0.5 else 0 for output_data in output]).to(device)
            correct += pred.eq(target.view_as(pred)).sum().item()

        val_accuracy = 100. * (correct / len(val_set))
        val_losses.append(loss.item())
        val_accs.append(val_accuracy)

        print('Epoch {} finished: train loss = {}, val loss = {}'.format(epoch + 1,
                                                                         train_losses[-1], val_losses[-1]))

In [37]:
import time
start_time = time.time()
train(10)
elapsed_time = time.time()-start_time
print(f'elapsed_time: {elapsed_time}')


Train Epoch: 1 [0/1800 (0%)]	Loss: 0.691277
Train Epoch: 1 [160/1800 (9%)]	Loss: 0.885873
Train Epoch: 1 [320/1800 (18%)]	Loss: 0.679173
Train Epoch: 1 [480/1800 (26%)]	Loss: 0.684840
Train Epoch: 1 [640/1800 (35%)]	Loss: 0.692103
Train Epoch: 1 [800/1800 (44%)]	Loss: 0.694850
Train Epoch: 1 [960/1800 (53%)]	Loss: 0.693868
Train Epoch: 1 [1120/1800 (61%)]	Loss: 0.595851
Train Epoch: 1 [1280/1800 (70%)]	Loss: 0.542146
Train Epoch: 1 [1440/1800 (79%)]	Loss: 0.592285
Train Epoch: 1 [1600/1800 (88%)]	Loss: 0.499671
Train Epoch: 1 [1760/1800 (96%)]	Loss: 0.505445
Train Epoch: 2 [0/1800 (0%)]	Loss: 0.618874
Train Epoch: 2 [160/1800 (9%)]	Loss: 0.477265
Train Epoch: 2 [320/1800 (18%)]	Loss: 0.419029
Train Epoch: 2 [480/1800 (26%)]	Loss: 0.599740
Train Epoch: 2 [640/1800 (35%)]	Loss: 0.492073
Train Epoch: 2 [800/1800 (44%)]	Loss: 0.440180
Train Epoch: 2 [960/1800 (53%)]	Loss: 0.507038
Train Epoch: 2 [1120/1800 (61%)]	Loss: 0.510532
Train Epoch: 2 [1280/1800 (70%)]	Loss: 0.493531
Train Epoch: 2

In [49]:
torch.save(model, './model.pt')


In [3]:
def test(model):
    model.eval()
    pred_list = []
    with torch.no_grad():
        for data in test_loader:
            data = data.to(device)
            output = model(data)
            pred = torch.tensor(
                [1 if output_data >= 0.5 else 0 for output_data in output]).to(device)
            pred_list.extend(pred)
    return pred_list

In [12]:
saved_model = torch.load("./model.pt", map_location=device)
answer = test(saved_model)

In [14]:
submission_df = pd.read_csv('./dataset/test/submission.csv')
submission_df.head()


,filename,label
0,image_001.png,NaN
1,image_002.png,NaN
2,image_003.png,NaN
3,image_004.png,NaN
4,image_005.png,NaN


In [23]:
for idx, pred in enumerate(answer):
    submission_df.loc[idx, "label"] = "normal" if (
        int(pred) == 0) else "covid"
submission_df.head()

,filename,label
0,image_001.png,covid
1,image_002.png,covid
2,image_003.png,normal
3,image_004.png,covid
4,image_005.png,normal


In [24]:
submission_df.to_csv("submission_result.csv", index=False)
